In [1]:
import os.path
import sys
import pickle
import re
import numpy as np
import spacy
import pandas as pd

<h2> Lematización <h/2>

In [3]:
def lematizacion(text):
    doc = nlp(text)
    processed_text = ' '.join([token.lemma_ for token in doc])
    return processed_text

nlp = spacy.load("es_core_news_sm")

# Proceso de tokenizacion y lematizacion  -----------------------------------------------------------------

df = pd.read_excel("Rest_Mex_2022.xlsx")

print(df)

df['Title_Opinion'] = df.apply(lambda row: f"{row['Title']} {row['Opinion']}", axis=1)
df = df.drop(['Title', 'Opinion'], axis=1)

df['Title_Opinion'] = df['Title_Opinion'].apply(lematizacion)

df.to_pickle('df_tokenizacion_lematizacion_final.pkl')

OSError: [E050] Can't find model 'es_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

<h2> Generación del lexicon de emojis <h/2>

In [19]:
# Especifica el nombre del archivo de Excel
nombre_archivo = 'Emojis lexicon.xlsx'

# Lee el archivo de Excel y crea un DataFrame
dataframe = pd.read_excel(nombre_archivo, header=None)  # No es necesario especificar sep='\t' para archivos de Excel

# Selecciona solo las columnas 2, 11 y 12 del DataFrame original
df_recortado = dataframe.iloc[:, [2, 11, 12]]

# Elimina la primera fila del DataFrame original
df = df.drop(df.index[0])

# Imprime el DataFrame recortado
print(df_recortado)

# Guarda el DataFrame recortado en un nuevo archivo de texto
df_recortado.to_csv("lexicon_emojis.txt", sep='\t', index=False)  # Ajusta el delimitador según tus necesidades

print(f"DataFrame exportado como lexicon_emojis.txt")

# Lee el archivo CSV especificando los nombres de las columnas
df = pd.read_csv('lexicon_emojis.txt', sep='\t', header=None, names=['emoji', 'negative', 'positive'])

# Elimina los duplicados basados en la columna 'emoji'
df_sin_duplicados = df.drop_duplicates('emoji')

# Crear un diccionario donde cada emoji tenga asociadas listas de tuplas (polaridad, valor)
diccionario_caracteristicas = {}

# Itera sobre las filas del DataFrame sin duplicados
for indice, fila in df_sin_duplicados.iterrows():
    emoji = fila['emoji']
    tupla_negative = ('negative', fila['negative'])
    tupla_positive = ('positive', fila['positive'])
    
    # Si el emoji ya está en el diccionario, actualiza las listas existentes
    if emoji in diccionario_caracteristicas:
        diccionario_caracteristicas[emoji].extend([tupla_negative, tupla_positive])
    else:
        diccionario_caracteristicas[emoji] = [tupla_negative, tupla_positive]

# Muestra el diccionario resultante
nombre_archivo_pickle = 'lexicon_emoji.pkl'

# Guarda el diccionario en el archivo pickle
with open(nombre_archivo_pickle, 'wb') as archivo_pickle:
    pickle.dump(diccionario_caracteristicas, archivo_pickle)

        2         11        12
0    emoji  negative  positive
1        😀  0.356135  0.976363
2        😃  0.385544  0.991735
3        😄  0.386087  0.885428
4        😁   0.37123  0.914501
..     ...       ...       ...
355      🔵  0.415303  1.015403
356      ⚫  0.400755  0.815105
357      ⚪  0.390018  0.947283
358      🏁  0.351811  0.985101
359   🏳️‍🌈  0.510269  1.481636

[360 rows x 3 columns]
DataFrame exportado como lexicon_emojis.txt


<h2> Generación del corpus final con los acumulados positivos y negativos <h/2>

In [10]:
def getSELFeatures(cadenas, lexicon_sel, lexicon_emoji):
    # Función para extraer características emocionales de las cadenas de texto
    features = []
    for cadena in cadenas:
        # Inicialización de valores de emociones y polaridades
        valor_alegria = 0.0
        valor_enojo = 0.0
        valor_miedo = 0.0
        valor_repulsion = 0.0
        valor_sorpresa = 0.0
        valor_tristeza = 0.0
        valor_positivo = 0.0
        valor_negativo = 0.0
        
        # Tokenización de la cadena de texto
        cadena_palabras = re.split('\s+', cadena)
        dic = {}
        
        # Procesamiento de cada palabra en la cadena
        for palabra in cadena_palabras:
            if palabra in lexicon_sel:
                # Si la palabra está en el lexicon_sel, se extraen características emocionales
                caracteristicas = lexicon_sel[palabra]
                for emocion, valor in caracteristicas:
                    # Acumulación de valores emocionales
                    if emocion == 'Alegría':
                        valor_alegria += float(valor)
                    elif emocion == 'Tristeza':
                        valor_tristeza += float(valor)
                    elif emocion == 'Enojo':
                        valor_enojo += float(valor)
                    elif emocion == 'Repulsión':
                        valor_repulsion += float(valor)
                    elif emocion == 'Miedo':
                        valor_miedo += float(valor)
                    elif emocion == 'Sorpresa':
                        valor_sorpresa += float(valor)

            if palabra in lexicon_emoji:
                # Si la palabra está en el lexicon_emoji, se extraen características de polaridad
                caracteristicas2 = lexicon_emoji[palabra]
                for polaridad, valor in caracteristicas2:
                    # Acumulación de valores de polaridad
                    if polaridad == 'positive':
                        valor_positivo = valor_positivo + float(valor)
                    elif polaridad == 'negative':
                        valor_negativo = valor_negativo + float(valor)

        # Almacenamiento de valores acumulados en un diccionario
        dic['_positive_'] = valor_positivo
        dic['_negative_'] = valor_negativo
        dic['_alegria_'] = valor_alegria
        dic['_tristeza_'] = valor_tristeza
        dic['_enojo_'] = valor_enojo
        dic['_repulsion_'] = valor_repulsion
        dic['_miedo_'] = valor_miedo
        dic['_sorpresa_'] = valor_sorpresa

        dic['acumuladopositivo'] = dic['_alegria_'] + dic['_sorpresa_'] + dic['_positive_']
        dic['acumuladonegative'] = dic['_enojo_'] + dic['_miedo_'] + dic['_repulsion_'] + dic['_tristeza_'] + dic['_negative_']

        # Agregación del diccionario a la lista de características
        features.append(dic)

    return features

def process_text(text):
    # Función para procesar texto usando spaCy
    doc = nlp(text)
    processed_text = ' '.join([token.lemma_ for token in doc if not token.is_stop])
    return processed_text

def process_row(row, lexicon_sel, lexicon_emoji):
    # Función para procesar una fila del DataFrame
    polaridad = getSELFeatures([row['Title_Opinion']], lexicon_sel, lexicon_emoji)
    return polaridad[0] if polaridad else None

"""# Carga del modelo de procesamiento de lenguaje natural (spaCy)
nlp = spacy.load("es_core_news_sm")"""

# Lectura del DataFrame desde un archivo pickle
df = pd.read_pickle("df_tokenizacion_lematizacion_final.pkl")

# Carga de lexicones desde archivos pickle
with open('lexicon_emoji.pkl', 'rb') as archivo_pickle:
    lexicon_emoji = pickle.load(archivo_pickle)

with open('lexicon_sel.pkl', 'rb') as archivo_pickle2:
    lexicon_sel = pickle.load(archivo_pickle2)

# Lista de emociones para iterar
emotions = ['_positive_', '_negative_', '_alegria_', '_tristeza_', '_enojo_', '_repulsion_', '_miedo_', '_sorpresa_', 'acumuladopositivo', 'acumuladonegative']

# Iteración sobre las emociones para agregar columnas al DataFrame
for emotion in emotions:
    df[emotion] = df['Title_Opinion'].apply(lambda x: process_row({'Title_Opinion': x}, lexicon_sel, lexicon_emoji)[emotion] if process_row({'Title_Opinion': x}, lexicon_sel, lexicon_emoji) else None)

# Eliminación de la columna 'Attraction'
df = df.drop(['Attraction'], axis=1)

# Guardado del DataFrame resultante en un archivo pickle
df.to_pickle('df_tokenizacion_lematizacion_emojis_final.pkl')

# Imprimir el DataFrame resultante
print(df)


       Polarity                                      Title_Opinion   
0             1  pésimo lugar pensar dos vez antes de ir a este...  \
1             1  no vayas a lugar de Eddie cuatro de yo ir reci...   
2             1  malo relación calidad-precio seguir corto y si...   
3             1  minusválido ? ¡ no tú alojser aquí ! al reserv...   
4             1  ser uno porquerio no perder su tiempo no perde...   
...         ...                                                ...   
30207         5  Verdadera joya arquitectónico ser uno construc...   
30208         5  Romántico mucho al estilo de Romeo y Julieta s...   
30209         5  parecer uno castillo Ideal para subir el escal...   
30210         5  Imperdible ser imperdible , de ahí poder ver m...   
30211         5  mucho bonito visto no tú poder ir de Guanajuat...   

       _positive_  _negative_  _alegria_  _tristeza_  _enojo_  _repulsion_   
0             0.0         0.0      0.792       1.129    1.392        0.630  \
1  